In [1]:
import PyPDF2 as ppy
from addClass import main as ac
from addStudent import main as ass
from addStudClass import main as asc
import pandas as pd
import textract
import pymysql

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.0.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
pdf = ppy.PdfFileReader("./Transcript for Robi.pdf")
pdf2 = ppy.PdfFileReader("UWUnofficialTranscript.pdf")
text = textract.process("UWUnofficialTranscript.pdf")
otherPDF = text.decode("utf-8")


In [9]:
data = getClasses(pdf2)
studentID = data['studentID']
data = data['data']
print(data)
print(studentID)

   Quarter Department ClassNum              ClassTitle Grade  Year
0   AUTUMN        CSE      142      COMPUTER PRGRMNG I   3.5  2016
1   AUTUMN        GEN      199          UNIV COMMUNITY    CR  2016
2   AUTUMN       INFO      200      INTELL FOUNDATIONS   3.2  2016
3   AUTUMN       PHIL      120          INTRO TO LOGIC   4.0  2016
4   WINTER       CLAS      430     GRK & ROM MYTHOLOGY   3.9  2017
5   WINTER       ENGL      131     COMPOSITN: EXPOSITN   3.7  2017
6   WINTER       MATH      126      CALC ANYL GEOM III   3.2  2017
7   WINTER       PHIL      335        PLATO'S REPUBLIC   3.9  2017
8   SPRING       ECON      200         INTRO MICROECON   3.8  2017
9   SPRING       MATH      308          MATRIX ALGEBRA   3.7  2017
10  SPRING       PHIL      240       W-INTRO TO ETHICS   3.7  2017
11  AUTUMN      ACCTG      215     ACCTG & FIN REPRTNG   4.0  2017
12  AUTUMN       ECON      201         INTRO MACROECON   3.7  2017
13  AUTUMN       JSIS      200   W-STATES & CAPITALISM   3.7  

In [6]:
def commitData(data, studentID):
    classes=data
#     studentID = data['studentID']
#     if (studentID == userID):     
    row = data.loc[0]
    classesBatch = []
    for i in range(len(data)):
        row = data.loc[i]
        classesBatch.append((row['Department'], row['ClassNum'], row['ClassTitle']))
    ac.main(classesBatch)
    
    row = data.loc[0]
    studClassBatch = []
    for i in range(len(data)):
        row = data.loc[i]
        studClassBatch.append((studentID, row['Quarter'], row['ClassNum'], row['Department'], row['ClassTitle'], row['Year']))
    asc.main(studClassBatch)

    

In [3]:
def mergePages(pages):
    breaks = []
    total = []
    # Look for the word 'Page' on each page 
    # assumptions: there is only one occurance
    # no valuable data preceds the occurance of 'Page'
    curQtr = ""
    studentNumber = ""
    for i in range(pages.getNumPages()):
        page = pages.getPage(i)
        parsed = page.extractText().split('\n')
        studentNumber = parsed[8]
        for i,x in enumerate(parsed):
            if "Page" in x:
                breaks = i
            if 'CURRENTLY ENROLLED' in x:
                curQtr = x.split('(')[1].split(')')[0].replace('QUARTER, ', "")        
        useful = parsed[breaks + 1: ]
        total.extend(useful)
    return {'total': total, 'curQtr': curQtr.strip(), 'studentID': studentNumber}

In [4]:
def getClasses(pdf):
    information = mergePages(pdf)
    parsed = information['total']
    curQtr = information['curQtr']
    studentID = information['studentID']
    durations = pd.DataFrame(columns = ['Quarter', 'Year', 'StatusWhile', 'index'])
    count = 0
    for i, x in enumerate(parsed):
        if 'AUTUMN' in x or 'WINTER' in x or 'SPRING' in x and 'D E S T R O Y' not in x:
            row = {'Quarter': x.split()[0], 'Year': x.split()[1], 'StatusWhile' : x.split()[2], 'index': i}
    #         y = x.split()
            durations = durations.append(row, ignore_index=True)
            count = count +1

    classes = pd.DataFrame(columns = ['Quarter', 'Department', 'ClassNum', 'ClassTitle', 'Grade'])
    for i in durations.iloc:
        ind = i['index'] + 1
        cur = parsed[ind]
        while 'QTR' not in cur and 'EARNED' not in cur and 'D E S T R O Y' not in cur and ind < len(parsed):
            if 'GEN ST' in cur:
                cur = cur.replace('GEN ST' ,"GEN")
            curSplit = cur.split()
            desc = ""
            grade =  curSplit[-1]
            for j in range(2, len(curSplit) - 2):
                desc = desc + " " + curSplit[j]
            if curQtr == str(i['Quarter'] + " " +i['Year']):
                desc = ""
                for j in range(2, len(curSplit) - 1):
                    desc = desc + " " + curSplit[j]
                grade = "IP"
            row = {'Quarter': i['Quarter'].strip(), 'Year': i['Year'].strip(), 'Grade': grade.strip(), 'Department': curSplit[0].strip(), 'ClassNum': curSplit[1].strip(), 'ClassTitle' : desc.strip() }
            classes = classes.append(row, ignore_index=True)
            ind = ind + 1
            cur = parsed[ind]
            
    return {'data': classes, 'studentID': studentID}

In [11]:
commitData(data, studentID)